In [ ]:
# SVMD test using the test signal from the paper 'A Novel Empirical Variational Mode Decomposition for Early Fault Feature Extraction'.

# Import modules
import numpy as np
import sys
sys.path.insert(0, '..')
from modules.tfa import get_rfftm, get_hilbert_af
from modules.tfa_svmd import svmd
from modules.plot import plot, subplots, plot_modes

In [ ]:
# Define signal
sr, du, du_extend = 12000, 1.0, 0.0
t = np.arange(int(sr*(du+2*du_extend)))/sr - du_extend

f1, f2, f3 = 20, 35, 210
y1, y2, y3 = 0.5*np.sin(2*np.pi*f1*t), 0.5*np.sin(2*np.pi*f2*t), 0.5*np.sin(2*np.pi*f3*t)
y3[:int(sr*(0.1+du_extend))] = 0
y3[int(sr*(0.2+du_extend)): int(sr*(0.8+du_extend))] = 0
y3[int(sr*(0.9+du_extend)):] = 0
y4 = np.zeros_like(t)
A0, C, T = 0.5, 750, 0.125
fr, fn = 8, 3000
A = 1 + A0*np.sin(2*np.pi*fr*t)
h = np.exp(-C*t)*np.sin(2*np.pi*fn*t)
pulse = np.roll(A*h, int(0.5*sr*T))
for i in range(1, 9):
    roll = int(i*sr*T-np.random.uniform(0.01*T, 0.02*T))
    #roll = int(i*sr*T)
    y4 += np.roll(pulse, roll) 

y = np.sum([y1,y2,y3,y4], axis=0)

In [ ]:
# Set parameters
out_thr, in_thr = 1e-5, 1e-10
out_iter_max, in_iter_max = 3, 150
alpha, beta = 1., 1e-3
return_type = 'modes, residual'

In [ ]:
# Decompose
y_rfft = get_rfftm(y)
Modes, res = svmd(y, sr, out_thr, in_thr, out_iter_max, in_iter_max, alpha, beta, return_type)
res_rfft = get_rfftm(res)
Modes_rfft = get_rfftm(Modes, axis=1)
am, fm = get_hilbert_af(Modes, sr, axis=1)

In [ ]:
# Plot
%matplotlib ipympl
plot(y_rfft, title='spectrum of the original signal', xlabel='frequency', ylabel='magnitude')
plot(res_rfft, title='spectrum of the svmd residual', xlabel='frequency', ylabel='magnitude')

plot_modes(np.array([y1, y2, y3]), t, y, y4, title='original signal and modes')
plot_modes(Modes, t, y, res, title='svmd decomposed modes')

subplots(Modes_rfft, np.arange(Modes_rfft.shape[1]), title='svmd modes rfft', \
         subtitle='mode', xlabel='frequency', ylabel='magnitude')
subplots(am, t, title='svmd instantaneous amplitude (envelope)', subtitle='mode', xlabel='time', ylabel='amplitude')
subplots(fm, t[:-1], title='svmd instantaneous frequency', subtitle='mode', xlabel='time', ylabel='frequency')